# Replicating [Yan et al. (2022)](https://arxiv.org/abs/2212.12372)

Our goal is to factor a number $N$ into two prime numbers $p$ and $q$. Yes, $N$ is selected so that this is always the case. Yan et al. (2022)'s claim is that, by using quantum computers to speed up classically-slow computations (specifically, finding smooth-relation pairs), then [Schnorr's classical factoring algorithm](https://eprint.iacr.org/2021/933) (which adopts a lattice-based approach to factoring) can be used to solve factoring with a "sublinear" number of qubits in the bit length of $N$.

The lattice used in Schnorr's algorithm is of dimension $n$, defined according to the bit length of $N$, which we'll call $m$. While Schnorr himself is a little coy on the exact formulation of his choice on $n$, Yan et al. (2022) suggest the dimension satisfies $n\sim2c\log N/\log\log N$, where $c$ is a lattice parameter close to $1$ that we'll meet in a moment.

Yan et al. (2022)'s method takes a relatively good [classical estimate solving the CVP](https://link.springer.com/article/10.1007/BF02579403), call it $b_{op}$, then uses QAOA to search in the unit hypercube on the lattice surrounding it, where "unit" is defined with consideration to the basis $D=[d_1,\dots,d_n]$ obtained by [lattice reduction](https://infoscience.epfl.ch/record/164484/files/nscan4.PDF).

So, all things considered, the implementation is structured as follows:

1. Firstly, we define the basis of the prime lattice $B_{n,c}\in\mathbb{R}^{(n+1)\times n}$, parameterised by the "precision parameter" $c$, and a target vector $t\in\mathbb{R}^{n+1}$ which embody the reduction of the prime factorisation problem to the CVP on a lattice, whose vectors are smooth-relation pairs (sr-pairs).
$$
    B_{n,c}=
\begin{pmatrix}
f(1)       & 0          & \cdots & 0      \\
0          & f(2)       & \cdots & 0      \\ 
\vdots     & \vdots     & \ddots & \vdots \\ 
0          & 0          & \cdots & f(n)   \\ 
N^c\ln p_1 & N^c\ln p_2 & \cdots & N^c\ln p_n
\end{pmatrix}
\phantom{\int}\text{and}\phantom{\int}
t=
\begin{pmatrix}
0 \\ 0 \\ \vdots \\ 0 \\ N^c\ln N
\end{pmatrix}
$$

2. Secondly, we perform lattice reduction (e.g. LLL) to obtain a reduced basis $D=[d_1,\dots,d_n]$ and corresponding Gram-Schmidt orthogonal basis $\tilde{D}=[\tilde{d}_1,\dots,\tilde{d}_n]$.

3. Thirdly, we obtain an approximate solution $b_{op}$ by rounding each of the Gram-Schmidt coefficients to give the coefficients $c_i=\lceil\mu_i\rfloor=\lceil\langle d_i,\tilde{d}_i\rangle/\langle\tilde{d}_i,\tilde{d}_i\rangle\rfloor$. That is,
$$
b_{op}=\sum_{i=1}^{n}c_id_i
$$

4. Then, we formulate an optimisation problem that looks for an even closer vector $v_{new}$ by floating each coefficient either $\pm1$ or $0$ (i.e. taking a unit step away or staying put for each lattice dimension). Hence, our optimisation problem is to find the set of floating variables $\{x_1,\dots,x_n\}$ that minimise the distance between $v_{new}$ and $t$;
$$
F(x_1,\dots,x_n)=\|t-v_{new}\|^2=\Bigg\|t-b_{op}+\sum_{i=1}^nx_id_i\Bigg\|^2.
$$

5. Next, this optimisation problem is mapped to a Hamiltonian by exchanging the floating variables $x_i$ for positive or negative Pauli-Z operators $\sigma_z^i$, whose measured eigenvalues correspond exactly with the classical optimisation problem. Denote $\hat{x}_i$ to be the Pauli-Z operator $\pm\sigma_z^i$ corresponding to up-floating or down-floating.
$$
H_P=\Bigg\|t-b_{op}+\sum_{i=1}^n\hat{x}_id_i\Bigg\|^2=\sum_{j=1}^{n+1}\Bigg|t_j-b^j_{op}+\sum_{i=1}^n\hat{x}_id_{i,j}\Bigg|^2.
$$

6. Jumping into the world of programming, we then build a [VQE](https://arxiv.org/abs/1304.3061) to minimise the expectation value of our Hamiltonian $H_P$ under the trial state $|\psi(\vec\theta)\rangle$ parameterised by $\vec\theta$. Yan et al. (2022) use QAOA, which is not convincing of its ability to provide quantum advantage.

## Problem Set-up

Decide on a number to be factored, given in binary.

In [1]:
# Imports as always...
import numpy as np
from fpylll import *
from copy import deepcopy

In [2]:
# Seed for random generation.
np.random.seed(42)

In [3]:
# No point computing them ourselves!
primes = [2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541,
 547,
 557,
 563,
 569,
 571,
 577,
 587,
 593,
 599,
 601,
 607,
 613,
 617,
 619,
 631,
 641,
 643,
 647,
 653,
 659,
 661,
 673,
 677,
 683,
 691,
 701,
 709,
 719,
 727,
 733,
 739,
 743,
 751,
 757,
 761,
 769,
 773,
 787,
 797,
 809,
 811,
 821,
 823,
 827,
 829,
 839,
 853,
 857,
 859,
 863,
 877,
 881,
 883,
 887,
 907,
 911,
 919,
 929,
 937,
 941,
 947,
 953,
 967,
 971,
 977,
 983,
 991,
 997,
 1009,
 1013,
 1019,
 1021,
 1031,
 1033,
 1039,
 1049,
 1051,
 1061,
 1063,
 1069,
 1087,
 1091,
 1093,
 1097,
 1103,
 1109,
 1117,
 1123,
 1129,
 1151,
 1153,
 1163,
 1171,
 1181,
 1187,
 1193,
 1201,
 1213,
 1217,
 1223,
 1229,
 1231,
 1237,
 1249,
 1259,
 1277,
 1279,
 1283,
 1289,
 1291,
 1297,
 1301,
 1303,
 1307,
 1319,
 1321,
 1327,
 1361,
 1367,
 1373,
 1381,
 1399,
 1409,
 1423,
 1427,
 1429,
 1433,
 1439,
 1447,
 1451,
 1453,
 1459,
 1471,
 1481,
 1483,
 1487,
 1489,
 1493,
 1499,
 1511,
 1523,
 1531,
 1543,
 1549,
 1553,
 1559,
 1567,
 1571,
 1579,
 1583,
 1597,
 1601,
 1607,
 1609,
 1613,
 1619,
 1621,
 1627,
 1637,
 1657,
 1663,
 1667,
 1669,
 1693,
 1697,
 1699,
 1709,
 1721,
 1723,
 1733,
 1741,
 1747,
 1753,
 1759,
 1777,
 1783,
 1787,
 1789,
 1801,
 1811,
 1823,
 1831,
 1847,
 1861,
 1867,
 1871,
 1873,
 1877,
 1879,
 1889,
 1901,
 1907,
 1913,
 1931,
 1933,
 1949,
 1951,
 1973,
 1979,
 1987,
 1993,
 1997,
 1999,
 2003,
 2011,
 2017,
 2027,
 2029,
 2039,
 2053,
 2063,
 2069,
 2081,
 2083,
 2087,
 2089,
 2099,
 2111,
 2113,
 2129,
 2131,
 2137,
 2141,
 2143,
 2153,
 2161,
 2179,
 2203,
 2207,
 2213,
 2221,
 2237,
 2239,
 2243,
 2251,
 2267,
 2269,
 2273,
 2281,
 2287,
 2293,
 2297,
 2309,
 2311,
 2333,
 2339,
 2341,
 2347,
 2351,
 2357,
 2371,
 2377,
 2381,
 2383,
 2389,
 2393,
 2399,
 2411,
 2417,
 2423,
 2437,
 2441,
 2447,
 2459,
 2467,
 2473,
 2477,
 2503,
 2521,
 2531,
 2539,
 2543,
 2549,
 2551,
 2557,
 2579,
 2591,
 2593,
 2609,
 2617,
 2621,
 2633,
 2647,
 2657,
 2659,
 2663,
 2671,
 2677,
 2683,
 2687,
 2689,
 2693,
 2699,
 2707,
 2711,
 2713,
 2719,
 2729,
 2731,
 2741,
 2749,
 2753,
 2767,
 2777,
 2789,
 2791,
 2797,
 2801,
 2803,
 2819,
 2833,
 2837,
 2843,
 2851,
 2857,
 2861,
 2879,
 2887,
 2897,
 2903,
 2909,
 2917,
 2927,
 2939,
 2953,
 2957,
 2963,
 2969,
 2971,
 2999,
 3001,
 3011,
 3019,
 3023,
 3037,
 3041,
 3049,
 3061,
 3067,
 3079,
 3083,
 3089,
 3109,
 3119,
 3121,
 3137,
 3163,
 3167,
 3169,
 3181,
 3187,
 3191,
 3203,
 3209,
 3217,
 3221,
 3229,
 3251,
 3253,
 3257,
 3259,
 3271,
 3299,
 3301,
 3307,
 3313,
 3319,
 3323,
 3329,
 3331,
 3343,
 3347,
 3359,
 3361,
 3371,
 3373,
 3389,
 3391,
 3407,
 3413,
 3433,
 3449,
 3457,
 3461,
 3463,
 3467,
 3469,
 3491,
 3499,
 3511,
 3517,
 3527,
 3529,
 3533,
 3539,
 3541,
 3547,
 3557,
 3559,
 3571,
 3581,
 3583,
 3593,
 3607,
 3613,
 3617,
 3623,
 3631,
 3637,
 3643,
 3659,
 3671,
 3673,
 3677,
 3691,
 3697,
 3701,
 3709,
 3719,
 3727,
 3733,
 3739,
 3761,
 3767,
 3769,
 3779,
 3793,
 3797,
 3803,
 3821,
 3823,
 3833,
 3847,
 3851,
 3853,
 3863,
 3877,
 3881,
 3889,
 3907,
 3911,
 3917,
 3919,
 3923,
 3929,
 3931,
 3943,
 3947,
 3967,
 3989,
 4001,
 4003,
 4007,
 4013,
 4019,
 4021,
 4027,
 4049,
 4051,
 4057,
 4073,
 4079,
 4091,
 4093,
 4099,
 4111,
 4127,
 4129,
 4133,
 4139,
 4153,
 4157,
 4159,
 4177,
 4201,
 4211,
 4217,
 4219,
 4229,
 4231,
 4241,
 4243,
 4253,
 4259,
 4261,
 4271,
 4273,
 4283,
 4289,
 4297,
 4327,
 4337,
 4339,
 4349,
 4357,
 4363,
 4373,
 4391,
 4397,
 4409,
 4421,
 4423,
 4441,
 4447,
 4451,
 4457,
 4463,
 4481,
 4483,
 4493,
 4507,
 4513,
 4517,
 4519,
 4523,
 4547,
 4549,
 4561,
 4567,
 4583,
 4591,
 4597,
 4603,
 4621,
 4637,
 4639,
 4643,
 4649,
 4651,
 4657,
 4663,
 4673,
 4679,
 4691,
 4703,
 4721,
 4723,
 4729,
 4733,
 4751,
 4759,
 4783,
 4787,
 4789,
 4793,
 4799,
 4801,
 4813,
 4817,
 4831,
 4861,
 4871,
 4877,
 4889,
 4903,
 4909,
 4919,
 4931,
 4933,
 4937,
 4943,
 4951,
 4957,
 4967,
 4969,
 4973,
 4987,
 4993,
 4999,
 5003,
 5009,
 5011,
 5021,
 5023,
 5039,
 5051,
 5059,
 5077,
 5081,
 5087,
 5099,
 5101,
 5107,
 5113,
 5119,
 5147,
 5153,
 5167,
 5171,
 5179,
 5189,
 5197,
 5209,
 5227,
 5231,
 5233,
 5237,
 5261,
 5273,
 5279,
 5281,
 5297,
 5303,
 5309,
 5323,
 5333,
 5347,
 5351,
 5381,
 5387,
 5393,
 5399,
 5407,
 5413,
 5417,
 5419,
 5431,
 5437,
 5441,
 5443,
 5449,
 5471,
 5477,
 5479,
 5483,
 5501,
 5503,
 5507,
 5519,
 5521,
 5527,
 5531,
 5557,
 5563,
 5569,
 5573,
 5581,
 5591,
 5623,
 5639,
 5641,
 5647,
 5651,
 5653,
 5657,
 5659,
 5669,
 5683,
 5689,
 5693,
 5701,
 5711,
 5717,
 5737,
 5741,
 5743,
 5749,
 5779,
 5783,
 5791,
 5801,
 5807,
 5813,
 5821,
 5827,
 5839,
 5843,
 5849,
 5851,
 5857,
 5861,
 5867,
 5869,
 5879,
 5881,
 5897,
 5903,
 5923,
 5927,
 5939,
 5953,
 5981,
 5987,
 6007,
 6011,
 6029,
 6037,
 6043,
 6047,
 6053,
 6067,
 6073,
 6079,
 6089,
 6091,
 6101,
 6113,
 6121,
 6131,
 6133,
 6143,
 6151,
 6163,
 6173,
 6197,
 6199,
 6203,
 6211,
 6217,
 6221,
 6229,
 6247,
 6257,
 6263,
 6269,
 6271,
 6277,
 6287,
 6299,
 6301,
 6311,
 6317,
 6323,
 6329,
 6337,
 6343,
 6353,
 6359,
 6361,
 6367,
 6373,
 6379,
 6389,
 6397,
 6421,
 6427,
 6449,
 6451,
 6469,
 6473,
 6481,
 6491,
 6521,
 6529,
 6547,
 6551,
 6553,
 6563,
 6569,
 6571,
 6577,
 6581,
 6599,
 6607,
 6619,
 6637,
 6653,
 6659,
 6661,
 6673,
 6679,
 6689,
 6691,
 6701,
 6703,
 6709,
 6719,
 6733,
 6737,
 6761,
 6763,
 6779,
 6781,
 6791,
 6793,
 6803,
 6823,
 6827,
 6829,
 6833,
 6841,
 6857,
 6863,
 6869,
 6871,
 6883,
 6899,
 6907,
 6911,
 6917,
 6947,
 6949,
 6959,
 6961,
 6967,
 6971,
 6977,
 6983,
 6991,
 6997,
 7001,
 7013,
 7019,
 7027,
 7039,
 7043,
 7057,
 7069,
 7079,
 7103,
 7109,
 7121,
 7127,
 7129,
 7151,
 7159,
 7177,
 7187,
 7193,
 7207,
 7211,
 7213,
 7219,
 7229,
 7237,
 7243,
 7247,
 7253,
 7283,
 7297,
 7307,
 7309,
 7321,
 7331,
 7333,
 7349,
 7351,
 7369,
 7393,
 7411,
 7417,
 7433,
 7451,
 7457,
 7459,
 7477,
 7481,
 7487,
 7489,
 7499,
 7507,
 7517,
 7523,
 7529,
 7537,
 7541,
 7547,
 7549,
 7559,
 7561,
 7573,
 7577,
 7583,
 7589,
 7591,
 7603,
 7607,
 7621,
 7639,
 7643,
 7649,
 7669,
 7673,
 7681,
 7687,
 7691,
 7699,
 7703,
 7717,
 7723,
 7727,
 7741,
 7753,
 7757,
 7759,
 7789,
 7793,
 7817,
 7823,
 7829,
 7841,
 7853,
 7867,
 7873,
 7877,
 7879,
 7883,
 7901,
 7907,
 7919]

In [4]:
# The integer to be factored.
p, q = primes[3], primes[8] 
N = p * q

# What is the bit-length of the integer?
N_bit_length = N.bit_length()

# Convert to binary (drop the '0b' prefix).
N_binary = bin(N)[2:]

print(f'Integer to be factored: N = {N} (into the factors p = {p} and q = {q}).')
print(f'This has bit-length {N_bit_length}')

Integer to be factored: N = 161 (into the factors p = 7 and q = 23).
This has bit-length 8


The lattice dimension $m$ is given by $m=\lfloor ln/\log n\rfloor$, where $n=\lfloor\log N\rfloor$ and $l$ is a hyperparamter that Yan et al. (2022) keep to $l\in\{1,2\}$.

In [5]:
# The claimed lattice dimension to factor this integer is l * log N / log log N.
l = 1

n = int(np.floor(np.log2(N)))
m = int(np.ceil(l * n / np.log2(n)))

print(f'The claim is that we need only a lattice of dimension {m}.')

The claim is that we need only a lattice of dimension 3.


The corresponding prime basis consists of the first $m$ prime numbers, along with -1.

In [6]:
# The corresponding prime basis.
prime_basis_set = set([-1] + primes[:m])

print(f'The corresponding prime basis is {prime_basis_set}.')

The corresponding prime basis is {2, 3, 5, -1}.


Constructing the prime lattice and target vector according to the integer to be factored. We do this according to appendix IV part A (page 13) of Yan et al. (2022). The hyperparameter $c$ is the "precision parameter".

In [23]:
c = 4

# Produce the random permutation for the diagonal.
f = np.random.permutation([(i + 1) // 2 for i in range(1, n + 1)])

# Create a zero matrix and add in the diagonal permutation.
B = np.zeros(shape=(n, n))
np.fill_diagonal(B, f)

# Create the extra final row and stick it on.
final_row = np.round(10 ** c * np.log(np.array(primes[:n])))
B = np.vstack((B, final_row))

# fpylll doesn't like numyp arrays, so convert it to a stnadard array.
B = [[int(b) for b in bs] for bs in B]

# Convert B to a matrix of integers (in fpylll's own type).
B = IntegerMatrix.from_matrix(B)
print(f'B = \n{B}')

# Define the target vector.
t = np.zeros(n + 1)
t[-1] = np.round(10 ** c * np.log(N))
t = tuple(t.astype(int))

# And again, if t could be a bunch of integers, that would be swell.
print(f'\nt = \n{t}')

B = 
[    3     0     0     0     0     0     0 ]
[    0     1     0     0     0     0     0 ]
[    0     0     2     0     0     0     0 ]
[    0     0     0     2     0     0     0 ]
[    0     0     0     0     1     0     0 ]
[    0     0     0     0     0     4     0 ]
[    0     0     0     0     0     0     3 ]
[ 6931 10986 16094 19459 23979 25649 28332 ]

t = 
(0, 0, 0, 0, 0, 0, 0, 50814)


## Babai's Algorithm: Finding an Approximate Solution $b_{op}$

Performing LLL lattice reduction to obtain a reduced basis $D$ and corresponding Gram-Schmidt orthogonal basis $\tilde{D}$.

Using the [fpylll](https://pypi.org/project/fpylll/) library, which is not supported on windows, so I am having to do this in the world's slowest virtual machine.

In [24]:
# LLL-reduction hyperparameter (using 0.75, according to Wikipedia)
delta = .75

D = deepcopy(B).transpose()
LLL.reduction(D, delta)
print(f'D (transposed) = \n{D}\n')

M = GSO.Mat(D, update=True)
w = M.babai(t)

A = IntegerMatrix(2 * D.nrows, D.ncols)
for i in range(D.nrows):
  for j in range(D.ncols):
    A[i, j] = D[i, j]

b = np.array(t)
for i in reversed(range(D.nrows)):
  for j in range(D.ncols):
    A[i + D.nrows, j] = int(b[j])
  b -= w[i] * np.array(D[i])

print("Which way is each coefficient rounded?")
M = GSO.Mat(A, update=True)
rounding_direction = []
for i in range(D.nrows):
  mu = M.get_mu(i + D.nrows, i)
  print(f'{mu=} c={w[i]}')
  rounding_direction.append(w[i] > mu)
  
b_op = np.array(D.multiply_left(w))
residual_vector = np.array(t) - b_op
print(f'\nb_op = \n{b_op}\n')
print(f'Hence, the residual vector is \n{residual_vector}\n')
print(f'This has a distance of {round(np.linalg.norm(residual_vector), 3)} to the target vector.')

D (transposed) = 
[  3  1  0  6 -2  0 -3  4 ]
[  0  2  0 -4  2  4 -6 -3 ]
[  3  5  2  0 -1 -4 -3 -5 ]
[  0  6  0  0 -5  4  3  2 ]
[  6  1 -2 -6  1  4  0  5 ]
[ -3 -7  8  2  0  0  0  2 ]
[  6 -5  4 -2  0  0  3 -7 ]

Which way is each coefficient rounded?
mu=328.6533333333333 c=329
mu=-2506.7481171548116 c=-2507
mu=864.4639736277535 c=864
mu=-527.5530563830364 c=-528
mu=3899.022288511351 c=3899
mu=2405.1790109593667 c=2405
mu=-3293.0463071338954 c=-3293

b_op = 
[    0    -4    -2     4     3     0     0 50817]

Hence, the residual vector is 
[ 0  4  2 -4 -3  0  0 -3]

This has a distance of 7.348 to the target vector.


In [25]:
# Reformatting after this step to make subsequent steps easier.
def integer_matrix_to_numpy(M):
  m, n = M.nrows, M.ncols
  D = np.zeros((m, n), dtype=int)
  M.to_matrix(D)
  return D

B = integer_matrix_to_numpy(B)
D = integer_matrix_to_numpy(D.transpose())
t = np.array(t)
rounding_direction = tuple(rounding_direction)

## The Problem Hamiltonian

The new vector $v_{new}$ to be optimally found is parameterised on $b_{op}$ in the following way:
$$
v_{new}=b_{op}+\sum_{i=1}^nx_id_i=\sum_{i=1}^n(c_i+x_i)d_i
$$

Naively, we may choose $x_i\in\{\pm1,0\}$ to step a unit amount in direction $d_i$ (or not move at all), which thus paints out the unit hypercube around $b_{op}=\sum_{i=1}^nc_id_i$ as our search space. However, we need not be this naive!

During the finding of $b_{op}$, the coefficients $c_i$ were found by rounding the Gram-Schmidt coefficients $\mu_i$ to the nearest integer. Hence, $b_{op}$ either overestimates or underestimates the $d_i$-th component. So when we are nudging $b_{op}$ around to yield $v_{new}$, there is no need to consider *further* over/under-estimation! That is, if $c_i$ is obtained through rounding $\mu_i$ upward, then using $x_i$ should not be $+1$ as that would overestimate *again* (and likewise for the rounding down case).

Consider the following:
$$
v_{new}=\sum_{i=1}^nc_id_i+\text{sign}(\mu_i-c_i)x_id_i=\sum_{i=1}^nc_id_i+\lceil \mu_i-c_i\rceil x_id_i
$$

Now, we may have $x_i\in\{0,1\}$ (giving us a pure QUBO formulation, as desired) which correspond to $n$ decisions about whether each dimension of the approximate solution $d_i$ was rounded the wrong way ($x_i=1$ indicates that $v_{new}$ tries rounding the other way, and $x_i=0$ indicates that $v_{new}$ agrees with the rounding of $b_{op}$ and nothing need be done).

Since $\text{sign}(\mu_i-c_i)=\lceil\mu_i-c_i\rceil=\pm1$, this additional coefficient gives us whether $b_{op}$ over- or under-estimated, thus allows us to make the restriction on which direction $v_{new}$ is able to 'step'. We have that $|\mu_i-c_i|<1$, thus $\lceil\mu_i-c_i\rceil=+1$ if $c_i$ is obtained by rounding down $\mu_i$ and $-1$ if by rounding up.

By keeping $x_i\in\{0,1\}$, we enforce that $v_{new}$ either step in the opposing direction to the rounding (i.e. round the other way on the coefficient $\mu_i$) or do nothing. If $x_i=1$, then $v_{vew}$'s $i$-th coefficient will be $\lceil\mu_i-c_i\rceil=\pm1$ step from $b_{op}$'s coefficient, corresponding to deciding to round $\mu_i$ to the *second*-nearest integer.

To complete the thought, the optimisation problem can now be formulated as a strictly binary optimisation problem;
$$
F(x_1,\dots,x_n)=\|t-v_{new}\|^2=\Bigg\|t-b_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil x_id_i\Bigg\|^2.
$$
and hence the Hamiltonian is obtained through a singular mapping;
$$
H_P=\Bigg\|t-b_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil\hat{x}_id_i\Bigg\|^2=\sum_{j=1}^{n+1}\Bigg|t_j-b^j_{op}+\sum_{i=1}^n\lceil\mu_i-c_i\rceil\hat{x}_id_{i,j}\Bigg|^2\phantom{\int}\text{with}\phantom{\int}\hat{x}_i=\frac{I-\sigma_z^i}{2}
$$

Note: the above ignores that $\mu_i$ may well have been negative, so each $\lceil\mu_i-c_i\rceil$ should really be $\lceil|\mu_i|-|c_i|\rceil$.

Let's define the Hamiltonian using `cirq`, which lets us do $I-\sigma_z^i$ very easily with `cirq.I(i) + -cirq.Z(i)`, where `i` refers to the $i$-th qubit and thus the tensor product is implicit. 

In [26]:
# More imports...
import cirq

In [27]:
# We have kept track of whether we rounded up, so convert this to +/-1 indications of whether each c_i is a rounding up or down.
# The direction v_new is allowed to step in each d_i is the opposite of this.
step_signs = - (np.array(rounding_direction).astype(int) * 2 - 1)

# Define the circuit.
circuit = cirq.LineQubit.range(D.shape[0])

# Add the appropriate operator to each qubit.
operators = []
for i, sign in zip(circuit, step_signs):
    operator = sign * ((cirq.I(i) + -cirq.Z(i)) / 2)
    operators.append(operator)
    
# Define the Hamiltonian.
H = cirq.PauliSum()
for j in range(D.shape[0]):
    h = residual_vector[j]
    for i in range(D.shape[1]):
        h = h + operators[i] * D[j, i]
    H = h ** 2
    
print(f'H = \n{H}')

H = 
34.0*I-4.0*Z(q(0))+6.0*Z(q(0))*Z(q(1))+10.0*Z(q(0))*Z(q(2))-4.0*Z(q(0))*Z(q(3))-10.0*Z(q(0))*Z(q(4))-4.0*Z(q(0))*Z(q(5))-14.0*Z(q(0))*Z(q(6))-3.0*Z(q(1))-5.0*Z(q(2))+2.0*Z(q(3))+5.0*Z(q(4))+2.0*Z(q(5))+7.0*Z(q(6))+7.5*Z(q(1))*Z(q(2))-3.0*Z(q(1))*Z(q(3))-7.5*Z(q(1))*Z(q(4))-3.0*Z(q(1))*Z(q(5))-10.5*Z(q(1))*Z(q(6))-5.0*Z(q(2))*Z(q(3))-12.5*Z(q(2))*Z(q(4))-5.0*Z(q(2))*Z(q(5))-17.5*Z(q(2))*Z(q(6))+5.0*Z(q(3))*Z(q(4))+2.0*Z(q(3))*Z(q(5))+7.0*Z(q(3))*Z(q(6))+5.0*Z(q(4))*Z(q(5))+17.5*Z(q(4))*Z(q(6))+7.0*Z(q(5))*Z(q(6))


## Sampling Low-energy Eigenstates of the Hamiltonian by QAOA

[This](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising) seems like a particularly relevant article from Google Quantum AI, which discusses QAOA for the Ising model (which our Hamiltonian is defined as) using Cirq.

Specifically, we have the binary optimisation problem $F(z_1,\dots,z_n)$, where each $z_i$ corresponds to the measurement outcome (i.e. an eigenvalue) of the Pauli-$Z$ operator on the $i$-th qubit (we're calling then $z_i$ rather than $x_i$ now because of this Pauli-$Z$ correlation -- it's just nice).

First, we prepare $n$ qubits in an equal superposition of all possible $z$ assignments; 

$$
H^{\otimes n}|0^n\rangle=\frac{1}{2^{n/2}}\sum_{z\in\{0,1\}^n}|z\rangle
$$

Now, our goal is to amplify the amplitdues of the $|z\rangle$ for which $F(z)$ is small, and suppress the amplitudes of those for which $F(z)$ is large. This will allow us to measure better choices of $z$ with higher probability.

Next, we act with the unitary $U(\gamma,F)=e^{i\pi\gamma F(Z)/2}$, where $\gamma$ is variational parameter, and $F(Z)$ denotes that we have exchanged the $z_i$ for Pauli-$Z$ operators, thus turning $F(z)$ (a real number) into $F(Z)$ (a matrix whose diagonal entries represent the possible values of $F(z)$). This sets the coefficients in the sum to be complex phases depending on $F$.

Following this, we act with the unitary operator $U(\beta,B)=e^{i\pi\beta B/2}$ (the "mixing" operation), where $\beta$ is another variational parameter, and $B=\sum_{i=1}^nX_i$ is the sum of Pauli-$X$ operator acting on each qubit. Each of these Pauli-$X$ operators commute, since they each apply to a different qubit, so we can rewrite this unitary as
$$
U(\beta,B)=\prod_{i=1}^ne^{i\pi\beta X_i/2}
$$

Since this operation is *not* diagonal in the computational basis (unlike the previous), there will be constructive and destructive interference, hopefully leading to enhancement of states corresponding to small values of $F$.

These two steps are repeatedly applied $p\geq1$ times, where $p$ here denotes the depth of the circuit. We allow $\gamma$ and $\beta$ to be chosen afresh each time (otherwise there wouldn't be much point in repeating it), giving a circuit that leaves our qubits in the state
$$
|\gamma,\beta\rangle=U(\beta_p,B)U(\gamma_p,F)\cdots U(\beta_1,B)U(\gamma_1,F)$
$$

It is our job to select $\gamma$ and $\beta$ so that the expectation value $F(\gamma,\beta)=\langle\gamma,\beta|F(Z)|\gamma,\beta\rangle$ is minimised so that our measurement of $|\gamma,\beta\rangle$ in the computational basis gives a good solution $z$ for $F(z)$ with high probability.

In our case, we have defined $F(Z)=H_P$.

### Defining the gamma unitary operations

Our $H_P$ was formulated as a two-dimensional Ising model, hence it is of the form $\sum_ih_iZ_i+\sum_{i,j}J_{i,j}Z_iZ_j$, where $h_i$ and $J_{i,j}$ are determined by the coefficients of the primary and quadratic terms of the QUBO problem.

Because $H_P$ is of this form, $e^{-i\gamma H_p}$ may be implemented as a product of the individual single qubit terms $Z_i^{\gamma h_i}$ and two qubit terms $(ZZ)_{ij}^{\gamma J_{i,j}}$.

In [40]:
# More imports...
import sympy
import qsimcirq
from scipy.optimize import minimize

In [36]:
# Building the circuit.
p = 1

# Get references to the qubits used in the Hamiltonian's definition.
qubits = H.qubits

# Sub-circuit implementing the gamma unitary operator for the i-th layer.
def generate_gamma_operator(i):
    # Define the gamma symbol (as a placeholder).
    gamma = sympy.Symbol(f'gamma_{i}')
    
    # Define the Pauli operators as dense Pauli strings (for recognising terms).
    Z = cirq.DensePauliString('Z')
    ZZ = cirq.DensePauliString('ZZ')
    I = cirq.DensePauliString('')
    
    # Consider each term in the Hamiltonian.
    for term in H:
        # Get the term's operator.
        term_operator = term.with_coefficient(1).gate
        
        # Determine which circuit translation to make, based on which operator this term has.
        if term_operator == Z:
            yield cirq.Z(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == ZZ:
            yield cirq.ZZ(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == I:
            yield []
            
        # If the term's operator is unrecognised, the Hamiltonian isn't quite right.
        else:
            raise Exception(f'Unrecognised term in H: {term}')
        
# Sub-circuit implementing the beta unitary ("mixing") operator for the i-th layer.
def generate_beta_operator(i):
    # Define the beta symbol (as a placeholder).
    beta = sympy.Symbol(f'beta_{i}')
    
    return [cirq.X(q) ** beta for q in qubits]

# Generate the circuit for QAOA.
qaoa_circuit = cirq.Circuit(
    # Preparation of uniform superposition.
    cirq.H.on_each(*qubits),
    
    # p layers of repeated U(gamma, H) and U(beta, B) operations.
    [(generate_gamma_operator(i), cirq.Moment(generate_beta_operator(i)),) for i in range(p)],
)

display(qaoa_circuit)

┌─────────────────────────────────┐   ┌─────────────────────────────────┐   ┌──────────────────────────────────┐   ┌──────────────────────────────────┐   ┌────────────────────────────────────────────────────┐   ┌───────────────────────────────────┐   ┌───────────────────────────────────────────────────┐   ┌──────────────────────────────────┐   ┌────────────────────────────────┐
0: ───H───Z^(-4.0*gamma_0)───ZZ──────────────────ZZ────────────────────────────────────ZZ────────────────────────────────────ZZ─────────────────────────────────────ZZ─────────────────────────────────────ZZ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^beta_0───
                             │                   │                                     │                                     │                                      │                                      │
1: ───H──────────────────────ZZ^(6.0*gamma_0)────┼────────────────Z^(-3.0*gamma_0)─────┼─────────────────────────────────────┼─────────────────ZZ───────────────────┼────────────────ZZ────────────────────┼─────────────────ZZ─────────────────────────────────────ZZ──────────────────────────────────────ZZ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^beta_0───
                                                 │                                     │                                     │                 │                    │                │                     │                 │                                      │                                       │
2: ───H──────────────────────────────────────────ZZ^(10.0*gamma_0)─────────────────────┼────────────────Z^(-5.0*gamma_0)─────┼─────────────────ZZ^(7.5*gamma_0)─────┼────────────────┼─────────────────────┼─────────────────┼────────────────ZZ────────────────────┼────────────────ZZ─────────────────────┼─────────────────ZZ────────────────────────────────────ZZ────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^beta_0───
                                                                                       │                                     │                                      │                │                     │                 │                │                     │                │                      │                 │                                     │
3: ───H────────────────────────────────────────────────────────────────────────────────ZZ^(-4.0*gamma_0)─────────────────────┼─────────────────Z^(2.0*gamma_0)──────┼────────────────ZZ^(-3.0*gamma_0)─────┼─────────────────┼────────────────ZZ^(-5.0*gamma_0)─────┼────────────────┼──────────────────────┼─────────────────┼────────────────ZZ───────────────────┼─────────────────ZZ───────────────────ZZ─────────────────────────────────────────────────────────────────────────X^beta_0───
                                                                                                                             │                                      │                                      │                 │                                      │                │                      │                 │                │                    │                 │                    │
4: ───H──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────ZZ^(-10.0*gamma_0)─────────────────────┼────────────────Z^(5.0*gamma_0)───────┼─────────────────ZZ^(-7.5*gamma_0)──────────────────────┼────────────────ZZ^(-12.5*gamma_0)─────┼─────────────────┼────────────────ZZ^(5.0*gamma_0)─────┼─────────────────┼────────────────────┼───────────────ZZ────────────────

In [38]:
# Finding the optimal parameters for the circuit.

# Get the parameters and observables out of the circuit.
parameter_names = sorted(cirq.parameter_names(qaoa_circuit))
observables = [term.with_coefficient(1) for term in H]

# Define the function to minimise.
def func_to_minimise(x):
    # Define a "parameter resolver" object, which we can use to assign values to parameters.
    parameter_resolver = cirq.ParamResolver(
        # Map each parameter to a value (i.e. assign the circuit's parameters).
        {parameter: value for parameter, value in zip(parameter_names, x)}
    )
    
    # Define a circuit simulator.
    simulator = qsimcirq.QSimSimulator(
        # Pass in some options for the simulator object -- this needs another object, it seems.
        qsimcirq.QSimOptions(
             # Use the GPU instead of the CPU.
             use_gpu=True,
             gpu_state_threads=512,
             verbosity=1
        )
    )
    
    # Simulate the expectation value.
    expectation = simulator.simulate_expectation_values(
        program=qaoa_circuit, observables=observables, param_resolver=parameter_resolver
    )
    
    # Compute the return.
    return sum(term.coefficient * value for term, value in zip(H, expectation)).real

# Let our initial guess be all zeros -- we don't know any better.
x_initial = np.asarray([.0] * len(parameter_names))

# Find the parameters values that minimise the expectation value.
optimal_parameter_values = minimize(func_to_minimise, x_initial, method='BFGS')

# Stick the parameter values in a parameter resolver object.
optimal_parameters = cirq.ParamResolver(
    {parameter: value for parameter, value in zip(parameter_names, optimal_parameter_values)}
)